# 데이터 불러오기

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook
import plotly.offline as pyo
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime
from datetime import date

In [ ]:
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
%matplotlib inline
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)  

In [ ]:
pd.set_option('display.max_columns',50) # 전체 열 출력하기
pd.set_option('display.max_rows',50) # 전체 행 출력하기

In [ ]:
# go to file
os.chdir('../DATA_raw2')

# 1. 멤버십 회원 구성 현황

In [ ]:
user = pd.read_pickle("../TBWA/user_fin2.pkl")
order = pd.read_pickle("../TBWA/order_fin.pkl")

In [ ]:
user.재가입횟수.describe()

count   1907989.0000
mean          0.4884
std           0.6980
min           0.0000
25%           0.0000
50%           0.0000
75%           1.0000
max          17.0000
Name: 재가입횟수, dtype: float64

## 1.1. 구매 경험 o vs 구매 경험 x 멤버 수

In [ ]:
demo = order.groupby("고유번호", as_index=False).agg({'구매일자' : 'nunique'})
demo.rename(columns = {'구매일자' : '구매횟수'}, inplace = True)
user = pd.merge(user, demo, on = '고유번호', how = 'left')
user['구매횟수'] = user['구매횟수'].fillna(value=0)

In [ ]:
def fun1(x) : 
    if x == 0:
        return '0회 구매'
    elif x == 1:
        return '1회 구매'
    elif x == 2:
        return '2회 구매'
    elif x == 3:
        return '3회 구매'
    elif x == 4:
        return '4회 구매'
    elif x == 5:
        return '5회 구매'
    elif x == 6:
        return '6회 구매'
    else :
        return '7회 이상 구매'
user['구매횟수2'] = user['구매횟수'].apply(fun1)

In [ ]:
df = user.groupby(['구매횟수2'], as_index = False).agg({'고유번호' : 'count'})
fig = px.pie(df, values='고유번호', names='구매횟수2', title='구매횟수별 멤버십 회원수', color = '구매횟수2') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

- 구매하지 않은 고객 : 54만명 존재. 
- 구매한 고객 중에선 1회 구매자가 많음

## 1.2 인구통계 현황

### 연령

In [ ]:
age = user.groupby(['연령대'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100

age['연령대2']='60세 이상'
age['연령대2'] = np.where(age['연령대']=='19세 미만', '10대 이하',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='20-24세')|(age['연령대']=='25-29세'), '20대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='30~34세')|(age['연령대']=='35~39세'), '30대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='40~44세')|(age['연령대']=='45~49세'), '40대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='50~54세')|(age['연령대']=='55~59세'), '50대',age['연령대2'])

In [ ]:
# 연령별 차이
# plotting
fig = px.bar(age, 
              x='연령대2', 
              y='퍼센트', 
              title='연령별 구성 - 2-30대 강세, 10대가 저조함', color="연령대", text_auto='.3s')
fig.update_layout(xaxis_title='연령대',
                  yaxis_title='퍼센트')

#fig.show()

In [ ]:
fig = px.pie(age, values='고유번호', names='연령대2', title='멤버십 연령대', color = '연령대2')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
age = user.groupby(['연령대2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대2'], ascending = True)
age['ratio'] = round(age.고유번호 / age.고유번호.sum() * 100,2)
age

### 성별

In [ ]:
sex = user.groupby(['성별'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['성별'], ascending = True)
sex['퍼센트'] = sex['고유번호'] / sex['고유번호'].sum() *100
sex

In [ ]:
fig = px.pie(sex, values='고유번호', names='성별', title='멤버십 내 성별비율', color = '성별', 
             color_discrete_map={ 'F': '#CD5C5C','M': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
user['연령대2']='60세 이상'
user['연령대2'] = np.where(user['연령대']=='19세 미만', '10대 이하',user['연령대2'])
user['연령대2'] = np.where((user['연령대']=='20-24세')|(user['연령대']=='25-29세'), '20대',user['연령대2'])
user['연령대2'] = np.where((user['연령대']=='30~34세')|(user['연령대']=='35~39세'), '30대',user['연령대2'])
user['연령대2'] = np.where((user['연령대']=='40~44세')|(user['연령대']=='45~49세'), '40대',user['연령대2'])
user['연령대2'] = np.where((user['연령대']=='50~54세')|(user['연령대']=='55~59세'), '50대',user['연령대2'])
#연령대별 성별
agesex = user.groupby(['연령대2','성별'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['연령대2'], ascending = True)
agesex

In [ ]:
#연령대별 성별
fig = px.bar(agesex, x="연령대2", y="고유번호", color="성별", color_discrete_map={
        'F': '#CD5C5C','M': '#4169E1'}, title="연령대별 성별 - 모든 연령대에서 여성이 강세, 십대는 반반",text_auto='.2s')
fig.show()

### 가입처

In [ ]:
sign = user.groupby(['가입처'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['고유번호'], ascending = False)
sign['퍼센트'] = sign['고유번호'] / sign['고유번호'].sum() *100
sign

In [ ]:
fig = px.pie(sign, values='고유번호', names='가입처', title='멤버십회원의 주 가입경로', color = '가입처') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# 2. 가입현황

In [ ]:
user2 = user.copy()

In [ ]:
user2['최초가입일2'] = user2['최초가입일'].astype(str)
user2['최초가입연월'] = user2['최초가입일2'].str[:7]

user2['최종가입일2'] = user2['최종가입일'].astype(str)
user2['최종가입연월'] = user2['최종가입일2'].str[:7]

user2['최초가입연도'] = user2['최초가입일'].dt.year
user2['최종가입연도'] = user2['최종가입일'].dt.year
user2['최초가입월'] = user2['최초가입일'].dt.month
user2['최종가입월'] = user2['최종가입일'].dt.month

## 연도별, 월별 가입 현황
- 2020년 기점으로 상승
- 월별 패턴은 없음

In [ ]:
log = user2.groupby(['최초가입연월'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['최초가입연월'], ascending = True).reset_index(drop=True)
log['퍼센트'] = log['고유번호'] / log['고유번호'].sum() *100
log['연도'] = log['최초가입연월'].str[:4].astype(int)
log['월'] = log['최초가입연월'].str[5:].astype(int)
log

In [ ]:
# 연도별, 연월별 가입 현황
# plotting
fig = px.line(log, 
              x='최초가입연월', 
              y='고유번호', 
              title='연월 별 가입현황, 변동 작다가 2020년 기점으로 상승')
fig.update_layout(xaxis_title='연월',
                  yaxis_title='카운트')
fig.show()

In [ ]:
log = log.groupby(['연도', '월'],as_index = False).agg({'고유번호':'sum'}).sort_values(by=['월'], ascending = True).reset_index(drop=True)
log['퍼센트'] = log['고유번호'] / log['고유번호'].sum() *100
log

In [ ]:
# 연도별, 연월별 가입 현황
# plotting
fig = px.line(log, 
              x='월', 
              y='고유번호', 
              title='월별 가입현황 - 패턴 x', color = '연도',
              color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(xaxis_title='월',
                  yaxis_title='카운트')
fig.show()

In [ ]:
# 2022년도만 월별 가입자 수
log[log.연도 == 2022][['연도', '월', '고유번호']]

In [ ]:
log = user2.groupby(['최초가입연도'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['최초가입연도'], ascending = True).reset_index(drop=True)
log['퍼센트'] = log['고유번호'] / log['고유번호'].sum() *100
log = log[:11]
log

In [ ]:
# plotting
fig = px.line(log, 
              x='최초가입연도', 
              y='고유번호', 
              title='연도별 가입현황')
fig.update_layout(xaxis_title='연도',
                  yaxis_title='카운트')
fig.show()

In [ ]:
df = user2.groupby(['최초가입연도'], as_index=False).agg({'고유번호' : 'count'})
fig = px.pie(df, values='고유번호', names='최초가입연도', title='최초가입연도 별 회원 수', color = '최초가입연도') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

- 2022년이 많이 나온것은 이전 연도에 탈퇴한 회원들은 잡히지 않고 있기 때문

## 가입주기

- 가입주기 : 회원들이 가입한 기간 / 그 기간동안 가입한 회원 수 = #분에 1명꼴로 가입한다. 

In [ ]:
# 가입주기 계산 
df = user2.sort_values(by=['최초가입일'], ascending = True)

df['직전가입일'] = df['최초가입일'].shift(1)
df['현재가입일'] = df['최초가입일']
df['가입소요시간'] = df['현재가입일'] - df['직전가입일']
df['가입소요시간'] = df['가입소요시간'].dt.days * 24 * 60
df['가입소요시간'].sum()/(len(df)-1)  #2.9분에 한명꼴로 !!!! 

2.936622242907188

In [ ]:
list1 = []
list2 = []

for i in df['최초가입연도'].unique() :
    data = df[df['최초가입연도'] == i]
    aa = data['가입소요시간'].sum()/(len(data)-1)
    list1.append(i)
    list2.append(aa)

In [ ]:
data = pd.DataFrame({'가입연도' : list1, '가입주기' : list2}).sort_values(by=['가입연도'], ascending = True).reset_index(drop=True)
data['가입연도'] = data['가입연도'].astype(str)
data = data[:11]
data

In [ ]:
# 가입 주기 
# plotting
fig = px.bar(data, 
              x='가입연도', 
              y='가입주기', 
              title='연도별 가입주기', color="가입연도", text_auto= '.2s')
fig.update_layout(xaxis_title='연도',
                  yaxis_title='주기')
fig.show()

## 가입자들의 유지기간 

In [ ]:
cc = pd.read_pickle('../DATA_raw2/도소매제거info.pkl')
## 쭉 유지회원
print('탈퇴하지않은 분들의 유지기간 : ', cc[cc.재가입횟수 == 0].유지기간2.mean()) #년

탈퇴하지않은 분들의 유지기간 :  1.8355746772992798


In [ ]:
## 재가입 회원
print('재가입하신 분들의 유지기간 : ', cc[cc.재가입횟수 > 0].유지기간2.mean()) #년

재가입하신 분들의 유지기간 :  1.5933288010343452


###  프로모션 시기에 가입(=최종가입)한 사람 전체 몇%?  = > 86.3 % 


In [ ]:
signup = pd.read_pickle("../TBWA/promo_signup.pkl")

In [ ]:
signup = signup[signup.고유번호.isin(user.고유번호.unique())]

In [ ]:
aa = []
for i in signup['프모기간가입'] : 
    aa.append(''.join(str(i)))
signup['프모'] = aa

In [ ]:
signup['프모'] = np.where(signup['프모'] =='[]', '행사x',signup['프모'])
signup['프모'] = np.where(signup['프모'] !='행사x', '행사o',signup['프모'])

signup['프모'].value_counts()

행사o    1646366
행사x     261623
Name: 프모, dtype: int64

In [ ]:
df = signup.groupby('프모', as_index = False).agg({'고유번호' : 'count'})
df['ratio'] = df.고유번호 / df.고유번호.sum() * 100
df

,프모,고유번호,ratio
0,행사o,1646366,86.2880
1,행사x,261623,13.7120


In [ ]:
df = signup.groupby('프모', as_index = False).agg({'고유번호' : 'count'})
fig = px.pie(df, values='고유번호', names='프모', title='가입기간에 프로모션이 열렸는가', color = '프모', 
             color_discrete_map={ '행사x': '#CD5C5C','행사o': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### 2022년 신규가입은 어떤 프로모션에서 가장 많이 일어났는가?

In [ ]:
df = pd.merge(user[['고유번호', '최초가입일','최종가입일', '재가입횟수']], signup[['고유번호','프모기간가입']], on = '고유번호', how = 'left')
df['최종가입연도'] = df.최종가입일.dt.year
df['최초가입연도'] = df.최초가입일.dt.year

In [ ]:
len(df[df.최초가입연도 == 2022] ) #2022년 신규가입 69만명

690797

In [ ]:
df = df[df.최초가입연도 == 2022].reset_index(drop=True) #2022년 신규가입
df.프모기간가입 = df.프모기간가입.astype(object)
df = df.explode('프모기간가입')
df['프모기간가입'] = df['프모기간가입'].fillna(value='행사x')

In [ ]:
df['프모기간가입'].value_counts().head(9) # 프로모션 번호

137    105826
138    105826
121     79996
122     79996
120     70709
130     58737
131     58737
127     58375
128     58375
Name: 프모기간가입, dtype: int64

In [ ]:
promo = pd.read_excel('file/ABC 프로모션 list2.xlsx')
promo = promo.sort_values(by=['시작일'], ascending = True).reset_index(drop=True)
promo = promo.reset_index()

# 3. 이탈 현황
- 이탈 연도를 다 알 수 없어서, 이탈 1회인 경우로 한정해서 진행

In [ ]:
def fun2(x) : 
    if x >= 5 :
        return '5회 이상'
    elif x == 4 : 
        return '4회'
    elif x ==3 : 
        return '3회'
    elif x ==2 : 
        return '2회'
    elif x ==1 : 
        return '1회'
    else :
        return '0회'
user2['재가입횟수2'] = user2['재가입횟수'].apply(fun2)

In [ ]:
df = user2.groupby(['재가입횟수2'], as_index=False).agg({'고유번호' : 'count'})
df['ratio'] = df.고유번호 / df.고유번호.sum() * 100
df

In [ ]:
df = user2.groupby(['재가입횟수2'], as_index=False).agg({'고유번호' : 'count'})
fig = px.pie(df, values='고유번호', names='재가입횟수2', title='전체 회원 내 이탈 경험 비율', color = '재가입횟수2') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
qq = user2[user2['재가입횟수'] >0]
df = qq.groupby(['재가입횟수2'], as_index=False).agg({'고유번호' : 'count'})
df['ratio'] = df.고유번호 / df.고유번호.sum() * 100
df

In [ ]:
qq = user2[user2['재가입횟수'] >0]
df = qq.groupby(['재가입횟수2'], as_index=False).agg({'고유번호' : 'count'})
fig = px.pie(df, values='고유번호', names='재가입횟수2', title='이탈자 내 경험 비율', color = '재가입횟수2') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
back1 = user2[user2['재가입횟수'] == 1] #전체의 28%
back2 = back1[back1['최종탈퇴일']!='모름']

back2['최종가입일'] = pd.to_datetime(back2['최종가입일'], errors='coerce')
back2['최종가입연월'] = back2['최종가입일'].astype(str)
back2['최종가입연월'] = back2['최종가입연월'].str[:7] 
back2['최종가입연도'] = back2['최종가입일'].dt.year
back2['최종가입월'] = back2['최종가입일'].dt.month

In [ ]:
print(len(user2[user2['재가입횟수'] == 1]) / len(user2))
back1 = user2[user2['재가입횟수'] == 1] #전체의 29%

0.2947123908995282


In [ ]:
back2 = back1[back1['최종탈퇴일']!='모름']

In [ ]:
back2['최종탈퇴일'] = pd.to_datetime(back2['최종탈퇴일'], errors='coerce')

back2['최종탈퇴연월'] = back2['최종탈퇴일'].astype(str)
back2['최종탈퇴연월'] = back2['최종탈퇴연월'].str[:7] 
back2['최종탈퇴연도'] = back2['최종탈퇴일'].dt.year
back2['최종탈퇴월'] = back2['최종탈퇴일'].dt.month

In [ ]:
back2 = back2.sort_values(by=['최종탈퇴일'], ascending = True)

In [ ]:
back2['최종탈퇴일'] = pd.to_datetime(back2['최종탈퇴일'], errors='coerce')
back2['최종탈퇴연도'] = back2['최종탈퇴일'].dt.year

### 연도별 이탈 현황
- 2017년 이후로 쭉 상승

In [ ]:
bb = back2.groupby(['최종탈퇴연도'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['최종탈퇴연도'], ascending = True).reset_index(drop=True)
bb['퍼센트'] = bb['고유번호'] / bb['고유번호'].sum() *100
bb = bb[:10]
bb

In [ ]:
fig = px.line(bb, 
              x='최종탈퇴연도', 
              y='고유번호', 
              title='연도별 이탈현황')
fig.update_layout(xaxis_title='연도',
                  yaxis_title='카운트')
fig.show()

In [ ]:
log = back2.groupby(['최종탈퇴연월'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['최종탈퇴연월'], ascending = True).reset_index(drop=True)
log['퍼센트'] = log['고유번호'] / log['고유번호'].sum() *100
log['연도'] = log['최종탈퇴연월'].str[:4].astype(int)
log['월'] = log['최종탈퇴연월'].str[5:].astype(int)
log = log[:129]
log

In [ ]:
# 연도별, 연월별 이탈 현황
# plotting
fig = px.line(log, 
              x='최종탈퇴연월', 
              y='고유번호', 
              title='연월 별 이탈현황 - 2015년 8월에 확 오름.')
fig.update_layout(xaxis_title='연월',
                  yaxis_title='카운트')
fig.show()

In [ ]:
# plotting
fig = px.line(log, 
              x='월', 
              y='고유번호', 
              title='월별 이탈현황 - 패턴 x', color = '연도',
              color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(xaxis_title='월',
                  yaxis_title='카운트')
fig.show()

## 이탈 주기

In [ ]:
# 가입주기 계산 2
df = back2.sort_values(by=['최종탈퇴일'], ascending = True)

df['직전탈퇴일'] = df['최종탈퇴일'].shift(1)
df['현재탈퇴일'] = df['최종탈퇴일']
df['탈퇴간격'] = df['현재탈퇴일'] - df['직전탈퇴일']
df['탈퇴간격'] = df['탈퇴간격'].dt.days * 24 * 60
df['탈퇴간격'].sum()/(len(df)-1)  #9.5분에 한명꼴로 !!!! -> 현실에선 이거보다 더 짧을수도. . .

9.561892148551344

In [ ]:
back2.columns

## 이탈자 특성

In [ ]:
#연령대별 성별
age = back2.groupby(['연령대'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['연령대'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age['연령대2']='60세 이상'
age['연령대2'] = np.where(age['연령대']=='19세 미만', '10대 이하',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='20-24세')|(age['연령대']=='25-29세'), '20대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='30~34세')|(age['연령대']=='35~39세'), '30대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='40~44세')|(age['연령대']=='45~49세'), '40대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='50~54세')|(age['연령대']=='55~59세'), '50대',age['연령대2'])


fig = px.bar(age, 
              x='연령대2', 
              y='퍼센트', 
              title='연령별 구성 - 멤버십 구성과 비슷함', color="연령대", text_auto='.3s')
fig.update_layout(xaxis_title='연령대',
                  yaxis_title='퍼센트')
fig.show()

In [ ]:
age = back2.groupby(['연령대2'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['연령대2'], ascending = True)
age['ratio'] = age.고유번호 / age.고유번호.sum() * 100
age

In [ ]:
agesex = back2.groupby(['연령대2','성별'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['연령대2'], ascending = True)
agesex['ratio'] = agesex.고유번호 / agesex.고유번호.sum() * 100
agesex

In [ ]:
agesex = back2.groupby(['연령대2','성별'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['연령대2'], ascending = True)


fig = px.bar(agesex, x="연령대2", y="고유번호", color="성별", color_discrete_map={
        'F': '#CD5C5C','M': '#4169E1'}, title="멤버십 구성이랑 별반 다를게 없음",text_auto='.2s')
fig.show()

In [ ]:
## 이탈자 특징
back1 = user2[user2['재가입횟수'] == 1]
back2 = back1[back1['최종탈퇴일']!='모름']
back2['최종탈퇴일'] = pd.to_datetime(back2['최종탈퇴일'], errors='coerce')
back2['유지기간'] = back2['최종탈퇴일'] - back2['최초가입일']
back2['유지기간'] = back2['유지기간'].dt.days / 30 /12
back2['유지기간'].mean()  #1.75년간 유지하다가 탈퇴

1.7593826199267704

# 4. 재가입 현황

## 재가입 비율
- 1회 가입한 사람 61%, 재가입한 사람 38%

In [ ]:
# 재가입 현황
print('재가입 고객 : ',len(user2[user2['재가입횟수'] > 0]) / len(user2) * 100 , '%')# 재가입으로 들어온 사람
print('재가입 고객 : ', len(user2[user2['재가입횟수'] > 0]) , '명' )
print('유지 고객 : ', len(user2[user2['재가입횟수'] == 0]) / len(user2) * 100, '%') # 꾸준히 사용하는 사람 
print('유지 고객 : ', len(user2[user2['재가입횟수'] == 0]) , '명' )

재가입 고객 :  38.469351762510165 %
재가입 고객 :  733991 명
유지 고객 :  61.530648237489835 %
유지 고객 :  1173998 명


In [ ]:
backlist = []
for i in range(len(user2)) : 
    if user2['재가입횟수'][i] > 0 :
        backlist.append('재가입 고객')
    else :
        backlist.append('유지 고객')

user2['재가입여부'] = backlist

In [ ]:
back = user2.groupby(['재가입여부'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['고유번호'], ascending = True).reset_index(drop=True)
fig = px.pie(back, values='고유번호', names='재가입여부', title='회원 내 재가입회원 비율', color = '재가입여부', 
             color_discrete_map={ '재가입 고객': '#CD5C5C','유지 고객': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## 재가입 횟수 현황

In [ ]:
list1 = user2['재가입횟수'].value_counts().index
list2 = user2['재가입횟수'].value_counts() 
data = pd.DataFrame({'재가입횟수':list1, '카운트' : list2})
data = data[1:]

In [ ]:
def back(x):
    if x>=3:
        return '3회 이상'
    elif x==2:
        return '2회'
    elif x==1:
        return '1회'
    else :
        return '0회'

data['횟수2'] = data['재가입횟수'].apply(back) 

In [ ]:
one = data.groupby('횟수2',as_index =False).agg({'카운트':'sum'})
one['ratio'] = one.카운트 / one.카운트.sum() * 100
one

In [ ]:
len(user2[user2.재가입횟수 > 0])

733991

## 연도별 재가입 수

In [ ]:
back1 = user2[user2['재가입횟수'] == 1] #전체의 28%
back2 = back1[back1['최종탈퇴일']!='모름']

back2['최종가입일'] = pd.to_datetime(back2['최종가입일'], errors='coerce')
back2['최종가입연월'] = back2['최종가입일'].astype(str)
back2['최종가입연월'] = back2['최종가입연월'].str[:7] 
back2['최종가입연도'] = back2['최종가입일'].dt.year
back2['최종가입월'] = back2['최종가입일'].dt.month

In [ ]:
user_re = user2[user2['재가입횟수'] > 0]

In [ ]:
log = user_re.groupby(['최종가입연월'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['최종가입연월'], ascending = True).reset_index(drop=True)
log['퍼센트'] = log['고유번호'] / log['고유번호'].sum() *100
log['연도'] = log['최종가입연월'].str[:4].astype(int)
log['월'] = log['최종가입연월'].str[5:].astype(int)
log = log[:102]
log

In [ ]:
fig = px.line(log, 
              x='최종가입연월', 
              y='고유번호', 
              title='연월 별 재가입현황.')
fig.update_layout(xaxis_title='연월',
                  yaxis_title='카운트')
fig.show()

In [ ]:
# plotting
fig = px.line(log, 
              x='월', 
              y='고유번호', 
              title='월별 재가입현황 - 패턴 x', color = '연도',
              color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(xaxis_title='월',
                  yaxis_title='카운트')
fig.show()

In [ ]:
# 2022년 월별 재가입수 테이블
log[log.연도 == 2022][['연도', '월', '고유번호']]

In [ ]:
bb = back2[back2['최종가입연도'] != 2023].groupby(['최종가입연도'],as_index = False).agg({'고유번호':'nunique'}).sort_values(by=['최종가입연도'], ascending = True).reset_index(drop=True)
bb['퍼센트'] = bb['고유번호'] / bb['고유번호'].sum() *100
fig = px.line(bb, 
              x='최종가입연도', 
              y='고유번호', 
              title='연도별 재가입현황')
fig.update_layout(xaxis_title='연도',
                  yaxis_title='카운트')
fig.show()

## 재가입 소요시간
- 평균 2.17년만에 돌아옴
- 재가입횟수가 늘어나면 늘어날수록 소요시간은 줄어들고 있음

In [ ]:
#재가입 소요시간 
user3 = user2[(user2['최종탈퇴일'] !='탈퇴X') | (user2['최종탈퇴일'] !='모름')]
user3['최종탈퇴일'] = pd.to_datetime(user3['최종탈퇴일'], errors='coerce')
user3['재가입주기'] = (user3['최종가입일'] - user3['최종탈퇴일'])
user3['재가입주기'] = user3['재가입주기'].dt.days / user3['재가입횟수']

In [ ]:
user3['재가입주기'].mean() / 30 / 12 #2.17년 만에 돌아온다. 

2.171709426927162

In [ ]:
aa= user3[user3.재가입횟수 == 1]
print('재가입횟수 1회 회원의 재가입 소요시간은' ,  round(aa.재가입주기.mean() / 30 / 12, 3), '년 입니다')

재가입횟수 1회 회원의 재가입 소요시간은 2.644 년 입니다


In [ ]:
bb= user3[user3.재가입횟수 == 2]
print('재가입횟수 1회 회원의 재가입 소요시간은' ,  round(bb.재가입주기.mean() / 30 / 12, 3), '년 입니다')

재가입횟수 1회 회원의 재가입 소요시간은 0.717 년 입니다


In [ ]:
cc= user3[user3.재가입횟수 >=3]
print('재가입횟수 1회 회원의 재가입 소요시간은' ,  round(cc.재가입주기.mean() / 30 / 12, 3), '년 입니다')

재가입횟수 1회 회원의 재가입 소요시간은 0.295 년 입니다


In [ ]:
user3['최종탈퇴연도'] = user3['최종탈퇴일'].dt.year

In [ ]:
back = user3.groupby(['최종탈퇴연도'],as_index = False).agg({'재가입주기':'mean'}).sort_values(by=['최종탈퇴연도'], ascending = True).reset_index(drop=True)
back['재가입주기2'] = back['재가입주기'] / 30 / 12 #년 수 
back

In [ ]:
# plotting
fig = px.bar(back, 
              x='최종탈퇴연도', 
              y='재가입주기2', 
              title='연도별 재가입주기 - 전체는 2.1년마다 재가입, 개별로는 2013년 탈퇴면 7년만에 재가입', text_auto= '.2s')
fig.update_layout(xaxis_title='연도',
                  yaxis_title='카운트')
fig.show()

### 재가입자 특징

In [ ]:
dateformat = "%Y-%m-%d"
str_datetime = "2023-02-03"
datetime_convert2 = datetime.strptime(str_datetime, dateformat)
datetime_convert2

datetime.datetime(2023, 2, 3, 0, 0)

### 재가입자 중 1회 구매 후 탈퇴한 자

In [ ]:
aa = order[order.고유번호.isin(user2[user2['재가입횟수'] >0 ].고유번호.unique())]
aa = pd.merge(aa, user2[['고유번호', '성별', '연령대','최종탈퇴일']], on='고유번호', how = 'left')

In [ ]:
temp2 = aa.groupby('고유번호', as_index= False).agg({'구매일자':'nunique'})
temp2.rename(columns = {'구매일자' : '구매횟수'}, inplace = True)

In [ ]:
temp = user[user.고유번호.isin(temp2[temp2.구매횟수 == 1].고유번호.unique())]

In [ ]:
print('재가입자 중 1회 구매후 탈퇴자는 재가입자 중 ', len(temp) / len(user2[user2.재가입횟수 >0 ]) * 100, '%이다')

재가입자 중 1회 구매후 탈퇴자는 재가입자 중  30.992750592309715 %이다


In [ ]:
# 성별
sex = temp.groupby(['성별'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['성별'], ascending = True)
fig = px.pie(sex, values='고유번호', names='성별', title='멤버십 내 성별비율', color = '성별', 
             color_discrete_map={ 'F': '#CD5C5C','M': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
sex = temp.groupby(['성별'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['성별'], ascending = True)
sex['ratio'] = sex.고유번호 / sex.고유번호.sum() * 100
sex

In [ ]:
age = temp.groupby(['연령대2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대2'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
age = temp.groupby(['연령대'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100

age['연령대2']='60세 이상'
age['연령대2'] = np.where(age['연령대']=='19세 미만', '10대 이하',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='20-24세')|(age['연령대']=='25-29세'), '20대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='30~34세')|(age['연령대']=='35~39세'), '30대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='40~44세')|(age['연령대']=='45~49세'), '40대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='50~54세')|(age['연령대']=='55~59세'), '50대',age['연령대2'])

In [ ]:
# 연령별 차이
# plotting
fig = px.bar(age, 
              x='연령대2', 
              y='퍼센트', 
              title='연령별 구성 - 2-30대 강세, 10대가 저조함', color="연령대", text_auto='.3s')
fig.update_layout(xaxis_title='연령대',
                  yaxis_title='퍼센트')

#fig.show()

In [ ]:
temp['c_PC로그인횟수(누적)'] = temp['PC로그인횟수_2020'] + temp['PC로그인횟수_2021'] + temp['PC로그인횟수_2022']
temp['c_모바일로그인횟수(누적)'] = temp['모바일로그인횟수_2020'] + temp['모바일로그인횟수_2021'] + temp['모바일로그인횟수_2022']
temp['c_앱로그인횟수(누적)'] = temp['앱로그인횟수_2020'] + temp['앱로그인횟수_2021'] + temp['앱로그인횟수_2022']
temp['총 로그인횟수'] = temp['c_PC로그인횟수(누적)'] + temp['c_모바일로그인횟수(누적)'] + temp['c_앱로그인횟수(누적)']
temp['c_년간포인트사용액(누적)'] = temp['년간포인트사용액_2020'] + temp['년간포인트사용액_2021'] + temp['년간포인트사용액_2022']
temp['c_출석체크_응모건수(누적)'] = temp['출석체크_응모건수_2020'] + temp['출석체크_응모건수_2021'] + temp['출석체크_응모건수_2022']
temp['c_출석체크_응모건수(누적)'] = temp['출석체크_응모건수_2020'] + temp['출석체크_응모건수_2021'] + temp['출석체크_응모건수_2022']
temp['c_출석체크_참여일수(누적)'] = temp['출석체크_참여일수_2020'] + temp['출석체크_참여일수_2021'] + temp['출석체크_참여일수_2022']
temp['c_일반이벤트_응모건수(누적)'] = temp['일반이벤트_응모건수_2020'] + temp['일반이벤트_응모건수_2021'] + temp['일반이벤트_응모건수_2022']
temp['c_드로우 응모건수(누적)'] = temp['드로우 응모건수_2020'] + temp['드로우 응모건수_2021'] + temp['드로우 응모건수_2022']

In [ ]:
temp = temp[['고유번호', '성별', '연령대', '주소', '가입처', '최초가입일', '최종가입일', '최종탈퇴일', 
        '리뷰작성건수', '구매횟수2','연령대2', 'c_PC로그인횟수(누적)', 'c_모바일로그인횟수(누적)', 'c_앱로그인횟수(누적)', '총 로그인횟수',
       'c_년간포인트사용액(누적)', 'c_출석체크_응모건수(누적)', 'c_출석체크_참여일수(누적)',
       'c_일반이벤트_응모건수(누적)', 'c_드로우 응모건수(누적)']]

In [ ]:
# 유저 특징
temp.describe().iloc[:,:]

In [ ]:
pd.set_option('display.max_columns',None) # 전체 열 출력하기
pd.set_option('display.max_rows',None) # 전체 행 출력하기

In [ ]:
cc = pd.read_pickle('../DATA_raw2/도소매제거info.pkl')
cc[cc.고유번호.isin(temp.고유번호.unique())][['유지기간2', '리뷰작성건수', 'c_PC로그인횟수(누적)', 'c_모바일로그인횟수(누적)',
       'c_앱로그인횟수(누적)', '총 로그인횟수', 'c_년간포인트사용액(누적)', 'c_출석체크_참여일수(누적)',
       'c_출석체크_응모건수(누적)', 'c_일반이벤트_응모건수(누적)', 'c_드로우 응모건수(누적)', '구매횟수',
       '연 평균 구매횟수', '구매상품수', '연 평균 구매상품수', '결제가 총합', '연 평균 결제가 총합', '쿠폰사용횟수',
       '연 평균 쿠폰사용횟수', '포인트사용횟수', '연 평균 포인트사용횟수', '연 평균 출첵참여횟수', '연 평균 일반이벤트횟수',
       '연 평균 드로우횟수', '연 평균 리뷰작성횟수', '신발구매주기', '비구매기간', '브랜드_c', '사이즈수_c']].describe().iloc[:,:]

In [ ]:
pd.set_option('display.max_columns',10) # 전체 열 출력하기
pd.set_option('display.max_rows',10) # 전체 행 출력하기

## [8p] 재가입한 회원과 그렇지 않은 회원(지속적으로 유지한 회원)의 구매건수 및 구매금액 비교


- 재가입횟수 1인 회원 안에서, 가입 이후 구매건수
- 재가입횟수 0인 회원 안에서, 구매건수
- 재가입 회원의 구매건수, 결제가 모두 큼 

In [ ]:
# 재가입횟수 0인 회원
temp = pd.read_pickle('../DATA_raw2/도소매제거info.pkl')
temp = temp[temp.고유번호.isin(user2[user2.재가입횟수 == 0].고유번호.unique())]

In [ ]:
print('평균 구매횟수', temp['구매횟수'].mean()) 
print('평균 구매상품수', temp['구매상품수'].mean()) 
print('평균 결제가 총합', temp['결제가 총합'].mean())

평균 구매횟수 1.7008010235111133
평균 구매상품수 2.4719803611249764
평균 결제가 총합 141977.73284111216


In [ ]:
print('평균 구매횟수', temp[temp.구매횟수 != 0]['구매횟수'].mean()) 
print('평균 구매상품수', temp[temp.구매횟수 != 0]['구매상품수'].mean()) 
print('평균 결제가 총합', temp[temp.구매횟수 != 0]['결제가 총합'].mean())

평균 구매횟수 2.477282793581789
평균 구매상품수 3.6005354712481963
평균 결제가 총합 206796.08594834612


In [ ]:
print('유지기간 동안 연 평균 구매횟수', temp['연 평균 구매횟수'].mean()) 
print('유지기간 동안 연 평균 구매상품수', temp['연 평균 구매상품수'].mean()) 
print('유지기간 동안 연 평균 결제가 총합', temp['연 평균 결제가 총합'].mean())

유지기간 동안 연 평균 구매횟수 0.8659976360684827
유지기간 동안 연 평균 구매상품수 1.2490593249310107
유지기간 동안 연 평균 결제가 총합 73409.3194930858


In [ ]:
def fun1(x) : 
    if x == 0:
        return '0회 구매'
    elif 1<=x<2:
        return '1회이상 2회미만 구매'
    elif 2<=x<3:
        return '2회이상 3회미만 구매'
    elif 3<=x< 4:
        return '3회이상 4회미만 구매'
    elif 4<=x <5:
        return '4회이상 5회미만 구매'
    elif 5<=x<6:
        return '5회이상 6회미만 구매'
    elif 6<=x<7:
        return '6회이상 7회미만 구매'
    else :
        return '7회 이상 구매'
temp['연평균구매횟수_c'] = temp['연 평균 구매횟수'].apply(fun1)

In [ ]:
#qq = temp.groupby(['연평균구매횟수_c'], as_index = False).agg({'고유번호' : 'count'})
#qq['ratio'] = qq.고유번호 / qq.고유번호.sum() * 100
#qq

In [ ]:
def fun2(x) : 
    if x == 0:
        return '0회 구매'
    elif x == 1:
        return '1회 구매'
    elif x==2:
        return '2회 구매'
    elif x==3:
        return '3회 구매'
    elif x== 4:
        return '4회 구매'
    elif x == 5:
        return '5회 구매'
    elif x == 6:
        return '6회 구매'
    else :
        return '7회 이상 구매'
temp['구매횟수_c'] = temp['구매횟수'].apply(fun2)

In [ ]:
qq = temp.groupby(['구매횟수_c'], as_index = False).agg({'고유번호' : 'count'})
qq['ratio'] = qq.고유번호 / qq.고유번호.sum() * 100
qq

In [ ]:
qq = temp[temp['구매횟수_c']!='0회 구매'].groupby(['구매횟수_c'], as_index = False).agg({'고유번호' : 'count'})
qq['ratio'] = qq.고유번호 / qq.고유번호.sum() * 100
qq

In [ ]:
# 재가입 회원
temp = pd.read_pickle('../DATA_raw2/도소매제거info.pkl')
temp = temp[temp.고유번호.isin(user2[user2.재가입횟수 >0].고유번호.unique())]

In [ ]:
print('평균 구매횟수', temp['구매횟수'].mean()) 
print('평균 구매상품수', temp['구매상품수'].mean()) 
print('평균 결제가 총합', temp['결제가 총합'].mean())

평균 구매횟수 1.9730732393176482
평균 구매상품수 2.7606346671825674
평균 결제가 총합 158225.48512175216


In [ ]:
print('평균 구매횟수', temp[temp.구매횟수 != 0]['구매횟수'].mean()) 
print('평균 구매상품수', temp[temp.구매횟수 != 0]['구매상품수'].mean()) 
print('평균 결제가 총합', temp[temp.구매횟수 != 0]['결제가 총합'].mean())

평균 구매횟수 2.5839764051379133
평균 구매상품수 3.6153826938894946
평균 결제가 총합 207215.27822631726


In [ ]:
print('유지기간 동안 연 평균 구매횟수', temp['연 평균 구매횟수'].mean()) 
print('유지기간 동안 연 평균 구매상품수', temp['연 평균 구매상품수'].mean()) 
print('유지기간 동안 연 평균 결제가 총합', temp['연 평균 결제가 총합'].mean())

유지기간 동안 연 평균 구매횟수 0.7839753422296286
유지기간 동안 연 평균 구매상품수 1.100885289193215
유지기간 동안 연 평균 결제가 총합 64823.0425814057


In [ ]:
temp['연평균구매횟수_c'] = temp['연 평균 구매횟수'].apply(fun1)

In [ ]:
temp['구매횟수_c'] = temp['구매횟수'].apply(fun2)
qq = temp.groupby(['구매횟수_c'], as_index = False).agg({'고유번호' : 'count'})
qq['ratio'] = qq.고유번호 / qq.고유번호.sum() * 100
qq

In [ ]:
qq = temp[temp['구매횟수_c']!='0회 구매'].groupby(['구매횟수_c'], as_index = False).agg({'고유번호' : 'count'})
qq['ratio'] = qq.고유번호 / qq.고유번호.sum()
qq

## 2022년 재가입시기 프로모션 확인

In [ ]:
signup

In [ ]:
df = pd.merge(user[['고유번호', '최초가입일', '최종가입일', '재가입횟수']], signup[['고유번호','프모기간가입']], on = '고유번호', how = 'left')
df['최종가입연도'] = df.최종가입일.dt.year
df['최초가입연도'] = df.최초가입일.dt.year

df = df[df.최초가입연도 == 2022].reset_index(drop=True) #2022년 신규가입
df.프모기간가입 = df.프모기간가입.astype(object)
df = df.explode('프모기간가입')
df['프모기간가입'] = df['프모기간가입'].fillna(value='행사x')

In [ ]:
df = pd.merge(user[['고유번호', '최종가입일', '재가입횟수']], signup[['고유번호','프모기간가입']], on = '고유번호', how = 'left')

In [ ]:
df['최종가입연도'] = df.최종가입일.dt.year
df = df[(df.재가입횟수 >=1 ) & (df.최종가입연도 == 2022)].reset_index(drop=True) # 2022년 재가입

In [ ]:
df.프모기간가입 = df.프모기간가입.astype(object)
df = df.explode('프모기간가입')
df['프모기간가입'] = df['프모기간가입'].fillna(value='행사x')

In [ ]:
df['프모기간가입'].value_counts().head(9)

121    50667
122    50667
138    49814
137    49814
120    41919
130    39470
131    39470
128    38512
127    38512
Name: 프모기간가입, dtype: int64